In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_columns',None)

# To get yearwise data

In [ ]:
def get_data(date):
    import bs4
    import requests
    import pandas as pd
    import numpy as np
    
    url = f'https://www.basketball-reference.com/leagues/NBA_{date}.html'
    data = requests.get(url)
    soup = bs4.BeautifulSoup(data.content)
    
    table1 = soup.find_all('table',id='per_game-team')
    table2 = soup.find_all('table',id='per_game-opponent')
    table3 = soup.find_all('table',id='totals-team')
    table4 = soup.find_all('table',id='totals-opponent')
    table5 = soup.find_all('table',id='per_poss-team')
    table6 = soup.find_all('table',id='per_poss-opponent')
    table7 = soup.find_all('table',id='advanced-team')

    
    cols = []
    for i in table1:
        for j in i.find_all('th',scope='col'):
            cols.append(j.text)
    
    cols_AS = []
    for i in table7:
        for j in i.find_all('th',scope='col'):
            cols_AS.append(j.text) 
            
    cols_AS = np.char.replace(cols_AS,'\xa0','null') 
        
    AS = pd.DataFrame(columns=cols_AS)
    PGS_team = pd.DataFrame(columns=cols)
    PGS_opponent = pd.DataFrame(columns=cols)
    TS_team = pd.DataFrame(columns=cols)
    TS_opponent = pd.DataFrame(columns=cols)
    PPS_team = pd.DataFrame(columns=cols)
    PPS_opponent = pd.DataFrame(columns=cols)
    
    for i in table1:
        row = []
        for j in i.find_all('tr'):
            row.append(j.text)
        row_len = len(row) - 2

    for i in table1: 
        for k in range(1,row_len+1):
            row = []
            for j in i.find_all('tr')[k]:
                if j.text=="" :
                    row.append(np.nan)
                else:
                    row.append(j.text)
            PGS_team.loc[k,:] = row
            
    for i in table2: 
        for k in range(1,row_len+1):
            row = []
            for j in i.find_all('tr')[k]:
                if j.text=="" :
                    row.append(np.nan)
                else:
                    row.append(j.text)
            PGS_opponent.loc[k,:] = row
            
    for i in table3: 
        for k in range(1,row_len+1):
            row = []
            for j in i.find_all('tr')[k]:
                if j.text=="" :
                    row.append(np.nan)
                else:
                    row.append(j.text)
            TS_team.loc[k,:] = row
            
    for i in table4: 
        for k in range(1,row_len+1):
            row = []
            for j in i.find_all('tr')[k]:
                if j.text=="" :
                    row.append(np.nan)
                else:
                    row.append(j.text)
            TS_opponent.loc[k,:] = row
                       
    for i in table5: 
        for k in range(1,row_len+1):
            row = []
            for j in i.find_all('tr')[k]:
                if j.text=="" :
                    row.append(np.nan)
                else:
                    row.append(j.text)
            PPS_team.loc[k,:] = row
            
    for i in table6: 
        for k in range(1,row_len+1):
            row = []
            for j in i.find_all('tr')[k]:
                if j.text=="" :
                    row.append(np.nan)
                else:
                    row.append(j.text)
            PPS_opponent.loc[k,:] = row
            
    for i in table7: 
            for k in range(2,row_len+2):
                row = []
                for j in i.find_all('tr')[k]:
                    if j.text=="" :
                        row.append(np.nan)
                    else:
                        row.append(j.text)             
                AS.loc[k,:] = row
                
    while 'null' in AS.columns:
        AS = AS.drop('null',axis=1)
        
    team = pd.merge(TS_team,PPS_team,on=['Team'],how='left').drop('Rk_y',axis=1)
    team = pd.merge(team,PGS_team,on=['Team'],how='left').drop('Rk',axis=1)
    
    opponent = pd.merge(TS_opponent,PGS_opponent,on=['Team'],how='left').drop('Rk_y',axis=1)
    opponent = pd.merge(opponent,PGS_opponent,on=['Team'],how='left').drop('Rk',axis=1)
    
    data = pd.concat([team,opponent],ignore_index=True).drop(['Rk_x','G','G_y'],axis=1)
            
    return data,AS

## To get the range of data

In [5]:
def get_range_data(x,y):
    import bs4
    import requests
    import pandas as pd
    team = pd.DataFrame(columns=['Team', 'G_x', 'MP_x', 'FG_x', 'FGA_x', 'FG%_x', '3P_x', '3PA_x',
       '3P%_x', '2P_x', '2PA_x', '2P%_x', 'FT_x', 'FTA_x', 'FT%_x', 'ORB_x',
       'DRB_x', 'TRB_x', 'AST_x', 'STL_x', 'BLK_x', 'TOV_x', 'PF_x', 'PTS_x',
       'MP_y', 'FG_y', 'FGA_y', 'FG%_y', '3P_y', '3PA_y', '3P%_y', '2P_y',
       '2PA_y', '2P%_y', 'FT_y', 'FTA_y', 'FT%_y', 'ORB_y', 'DRB_y', 'TRB_y',
       'AST_y', 'STL_y', 'BLK_y', 'TOV_y', 'PF_y', 'PTS_y', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'])
    for i in range(x,y+1):
        team1,AS = get_data(i)
        team = pd.concat([team,team1],ignore_index=True)
    return team